In [1]:
from datasets import load_from_disk


all_data = []
ds_DBAR = load_from_disk("/home/lesong/codes/SimPO/datasets/llama3.1_8B_ultrafeedback/cpo_dataset_1")
for item in ds_DBAR["train"]:
    all_data.append({
        "prompt": item["prompt"],
        "chosen": item["chosen"],
        "rejected": item["rejected"],
        "type": "DBAR"
    })

/home/lesong/anaconda3/envs/simpo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
def add_prefix(x):
    x["prompt"] = "tmp" + x["prompt"]
    return x
ds_DBAR['train'] = ds_DBAR['train'].map(lambda x: add_prefix(x))

In [19]:
ds_DBAR['train'][0]

{'prompt': "tmpIn this task, you're given reviews of various products in one of these languages 1) English 2) Japanese 3) German 4) French 5) Chinese 6) Spanish. Given a review you need to predict whether the language of the review is english or non-english\n\nwent through 3 in one day doesn't fit correct and couldn't get bubbles out (better without)",
 'chosen': [{'content': "In this task, you're given reviews of various products in one of these languages 1) English 2) Japanese 3) German 4) French 5) Chinese 6) Spanish. Given a review you need to predict whether the language of the review is english or non-english\n\nwent through 3 in one day doesn't fit correct and couldn't get bubbles out (better without)",
   'role': 'user'},
  {'content': 'I\'ll do my best to predict the language of the given review. Based on the review "went through 3 in one day doesn\'t fit correct and couldn\'t get bubbles out (better without)", I would predict that the language of the review is **English**.',


In [4]:
import datasets
import jsonlines


output_data = []
with jsonlines.open("/home/lesong/codes/SimPO/datasets/llama3.1_8B_ultrafeedback/all_train_data_1.json") as reader:
    for obj in reader:
        output_data.append(obj)

dataset = datasets.Dataset.from_list(output_data)
dataset = dataset.train_test_split(test_size=0.2)
dataset.save_to_disk("/home/lesong/codes/SimPO/datasets/llama3.1_8B_ultrafeedback/cpo_dataset")

Saving the dataset (1/1 shards): 100%|██████████| 2667/2667 [00:00<00:00, 68377.45 examples/s]


In [9]:
import jsonlines

with jsonlines.open("./on_policy_data_gen/datasets/llama3.1_8B_ultrafeedback/cpo_data.json") as reader:
    for item in reader:
        all_data.append({
        "prompt": item["prompt"],
        "chosen": item["chosen"],
        "rejected": item["rejected"],
        "type": "D"
    })

In [12]:
with jsonlines.open("all_train_data.json", "w") as writter:
    writter.write_all(all_data)

In [2]:
import json
import jsonlines
from vllm import LLM, SamplingParams
from datasets import load_dataset, load_from_disk
import os
import json





llm = LLM(model=args.model)
tokenizer = llm.get_tokenizer()

train_dataset= load_dataset(data_dir, split='train_prefs')

prompts = sorted(list(set(train_dataset['prompt'])))

conversations = [tokenizer.apply_chat_template([{'role': 'user', 'content': prompt}], tokenize=False, add_generation_prompt=True) for prompt in prompts]

sampling_params = SamplingParams(temperature=args.temperature, 
                                 top_p=args.top_p, 
                                 max_tokens=args.max_tokens, 
                                 seed=args.seed,)
outputs = llm.generate(conversations, sampling_params)

# Save the outputs as a JSON file.
output_data = []
for i, output in enumerate(outputs):
    prompt = output.prompt
    generated_text = output.outputs[0].text
    output_data.append({
        'prompt': prompts[i],
        "format_prompt": prompt,
        'generated_text': generated_text,
    })


all_data = []
with jsonlines.open("./on_policy_data_gen/datasets/llama3.1_8B_ultrafeedback/cpo_data.json") as reader:
    for obj in reader:
        obj["type"] = "D"
        obj["rejected"][1]["content"] = 
        print(obj)
        break

{'prompt': 'Your task is to decide whether question decomposition is necessary to answer a given question, if yes, which sub-question shall be asked. \n    Do not ask same or similar sub-questions that you have asked before.\n    Decompose the question in a reasonable and smart way. \n    Only give one sub-question each time.\n    Sub questions that you have asked before, if any:\n                \n                \n    Think step by step.\n    Based on the given context, provide your answer to the following question: What is the river where Long Spruce Generating Station is located a tributary of?\n    Your answer shall be in the following JSON format:\n    {\n        "decomposition": true, \n        "sub question": <A sub question that you would like to ask>\n    }\n    OR\n    {"decomposition": false}\n    ', 'chosen': [{'content': 'Your task is to decide whether question decomposition is necessary to answer a given question, if yes, which sub-question shall be asked. \n    Do not a